# LCEL అడ్వాన్స్డ్ ఫీచర్స్

ఈ ట్యుటోరియల్‌లో, మనం LangChain Expression Language (LCEL) యొక్క అడ్వాన్స్డ్ ఫీచర్స్ గురించి నేర్చుకుంటాము. ఇవి మరింత కాంప్లెక్స్ LLM అప్లికేషన్‌లను బిల్డ్ చేయడానికి సహాయపడతాయి.

## సెటప్

మొదట, మనం అవసరమైన లైబ్రరీలను ఇన్‌స్టాల్ చేసుకుందాం:

In [ ]:
# అవసరమైన లైబ్రరీలను ఇన్‌స్టాల్ చేయడం
!pip install langchain langchain-openai openai

ఇప్పుడు, మనం OpenAI API కీని సెట్ చేద్దాం:

In [ ]:
import os
from dotenv import load_dotenv

# .env ఫైల్ నుండి API కీని లోడ్ చేయడం
load_dotenv()

# లేదా డైరెక్ట్‌గా సెట్ చేయడం (డెవలప్‌మెంట్ కోసం మాత్రమే, ప్రొడక్షన్‌లో ఉపయోగించవద్దు)
# os.environ["OPENAI_API_KEY"] = "మీ-API-కీ-ఇక్కడ-పెట్టండి"

## LCEL తో మెమరీ ఇంటిగ్రేషన్

LCEL మనకు మెమరీని ఇంటిగ్రేట్ చేయడానికి అనుమతిస్తుంది, ఇది కన్వర్సేషనల్ అప్లికేషన్‌లకు ఉపయోగపడుతుంది:

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain.memory import ConversationBufferMemory

# ప్రాంప్ట్ టెంప్లేట్ క్రియేట్ చేయడం
prompt = ChatPromptTemplate.from_messages([
    ("system", "మీరు తెలుగు క్రికెట్ నిపుణుడు. IPL మరియు తెలుగు క్రికెట్ ఆటగాళ్ల గురించి సమాచారం అందించండి."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# LLM క్రియేట్ చేయడం
llm = ChatOpenAI(model="gpt-3.5-turbo")

# మెమరీ క్రియేట్ చేయడం
memory = ConversationBufferMemory(return_messages=True, memory_key="history")

# LCEL ఉపయోగించి పైప్‌లైన్ క్రియేట్ చేయడం
chain = {
    "input": lambda x: x["input"],
    "history": memory.load_memory_variables
} | prompt | llm

# కన్వర్సేషన్ స్టార్ట్ చేయడం
response = chain.invoke({"input": "IPL 2023లో సన్‌రైజర్స్ హైదరాబాద్ ఎలా పర్ఫార్మ్ చేసింది?"})
print(response.content)

# మెమరీకి కన్వర్సేషన్‌ని సేవ్ చేయడం
memory.save_context({"input": "IPL 2023లో సన్‌రైజర్స్ హైదరాబాద్ ఎలా పర్ఫార్మ్ చేసింది?"}, {"output": response.content})

# కన్వర్సేషన్ కొనసాగించడం
response = chain.invoke({"input": "వారి టాప్ బ్యాట్స్‌మెన్ ఎవరు?"})
print(response.content)

# మెమరీకి కన్వర్సేషన్‌ని సేవ్ చేయడం
memory.save_context({"input": "వారి టాప్ బ్యాట్స్‌మెన్ ఎవరు?"}, {"output": response.content})

## LCEL తో కస్టమ్ కంపోనెంట్స్

LCEL మనకు కస్టమ్ కంపోనెంట్స్‌ని క్రియేట్ చేయడానికి అనుమతిస్తుంది:

In [ ]:
from langchain_core.runnables import Runnable
from typing import Dict, Any

# తెలుగు సినిమా డేటాబేస్ క్లాస్
class TeluguMovieDatabase(Runnable):
    """తెలుగు సినిమాల సమాచారాన్ని అందించే కస్టమ్ కంపోనెంట్."""
    
    def __init__(self):
        # సింపుల్ డేటాబేస్
        self.movies = {
            "బాహుబలి": {
                "పూర్తి పేరు": "బాహుబలి: ది బిగినింగ్",
                "సంవత్సరం": 2015,
                "దర్శకుడు": "ఎస్.ఎస్. రాజమౌళి",
                "ప్రధాన నటులు": ["ప్రభాస్", "రానా దగ్గుబాటి", "అనుష్క శెట్టి", "తమన్నా భాటియా"],
                "బాక్సాఫీస్": "₹650 కోట్లు"
            },
            "RRR": {
                "పూర్తి పేరు": "రౌద్రం రణం రుధిరం",
                "సంవత్సరం": 2022,
                "దర్శకుడు": "ఎస్.ఎస్. రాజమౌళి",
                "ప్రధాన నటులు": ["రామ్ చరణ్", "ఎన్.టి.ఆర్", "ఆలియా భట్", "అజయ్ దేవగన్"],
                "బాక్సాఫీస్": "₹1,200 కోట్లు",
                "అవార్డులు": ["ఆస్కార్ - బెస్ట్ ఒరిజినల్ సాంగ్"]
            },
            "పుష్ప": {
                "పూర్తి పేరు": "పుష్ప: ది రైజ్",
                "సంవత్సరం": 2021,
                "దర్శకుడు": "సుకుమార్",
                "ప్రధాన నటులు": ["అల్లు అర్జున్", "రష్మిక మందన్నా", "ఫహద్ ఫాసిల్"],
                "బాక్సాఫీస్": "₹350 కోట్లు"
            }
        }
    
    def invoke(self, input: Dict[str, Any]) -> Dict[str, Any]:
        """సినిమా పేరు ఆధారంగా సమాచారాన్ని రిటర్న్ చేస్తుంది."""
        movie_name = input.get("movie_name", "")
        if movie_name in self.movies:
            return {"movie_info": self.movies[movie_name]}
        else:
            return {"movie_info": f"'{movie_name}' సినిమా గురించి సమాచారం అందుబాటులో లేదు."}

# కస్టమ్ కంపోనెంట్‌ని ఉపయోగించడం
movie_db = TeluguMovieDatabase()
result = movie_db.invoke({"movie_name": "RRR"})
print(result)

ఇప్పుడు, మనం ఈ కస్టమ్ కంపోనెంట్‌ని LCEL పైప్‌లైన్‌లో ఉపయోగిద్దాం:

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ప్రాంప్ట్ టెంప్లేట్ క్రియేట్ చేయడం
prompt = PromptTemplate.from_template(
    """కింది తెలుగు సినిమా సమాచారం ఆధారంగా, ఆ సినిమా గురించి ఒక చిన్న సమీక్ష రాయండి:
    
    సినిమా సమాచారం: {movie_info}
    
    సమీక్ష:
    """
)

# LLM క్రియేట్ చేయడం
llm = ChatOpenAI(model="gpt-3.5-turbo")

# LCEL ఉపయోగించి పైప్‌లైన్ క్రియేట్ చేయడం
chain = {"movie_name": lambda x: x["movie_name"]} | movie_db | prompt | llm | StrOutputParser()

# పైప్‌లైన్‌ని రన్ చేయడం
response = chain.invoke({"movie_name": "బాహుబలి"})
print(response)

## LCEL తో పారాలలైజేషన్

LCEL మనకు పారాలెల్‌గా టాస్క్‌లను రన్ చేయడానికి అనుమతిస్తుంది:

In [ ]:
from langchain_core.runnables import RunnableParallel

# ప్రాంప్ట్ టెంప్లేట్స్ క్రియేట్ చేయడం
summary_prompt = PromptTemplate.from_template(
    "తెలుగు సినిమా {movie_name} కథ సారాంశం 2-3 వాక్యాలలో రాయండి."
)

review_prompt = PromptTemplate.from_template(
    "తెలుగు సినిమా {movie_name} గురించి మీ అభిప్రాయం 2-3 వాక్యాలలో తెలపండి."
)

rating_prompt = PromptTemplate.from_template(
    "తెలుగు సినిమా {movie_name}కి 10లో ఎన్ని పాయింట్లు ఇస్తారు? కారణాలతో సహా తెలపండి."
)

# LLM క్రియేట్ చేయడం
llm = ChatOpenAI(model="gpt-3.5-turbo")

# ఔట్‌పుట్ పార్సర్ క్రియేట్ చేయడం
output_parser = StrOutputParser()

# ఇండివిడ్యువల్ చెయిన్స్ క్రియేట్ చేయడం
summary_chain = summary_prompt | llm | output_parser
review_chain = review_prompt | llm | output_parser
rating_chain = rating_prompt | llm | output_parser

# పారాలెల్ రన్నబుల్ క్రియేట్ చేయడం
parallel_chain = RunnableParallel(
    summary=summary_chain,
    review=review_chain,
    rating=rating_chain
)

# పారాలెల్‌గా రన్ చేయడం
results = parallel_chain.invoke({"movie_name": "పుష్ప"})

print("సారాంశం:")
print(results["summary"])
print("\nసమీక్ష:")
print(results["review"])
print("\nరేటింగ్:")
print(results["rating"])

## LCEL తో కండిషనల్ లాజిక్

LCEL మనకు కండిషనల్ లాజిక్‌ని ఇంప్లిమెంట్ చేయడానికి అనుమతిస్తుంది:

In [ ]:
from langchain_core.runnables import RunnableBranch

# ప్రాంప్ట్ టెంప్లేట్స్ క్రియేట్ చేయడం
cricket_prompt = PromptTemplate.from_template(
    "క్రికెట్ ఆటగాడు {person_name} గురించి ఒక చిన్న బయోగ్రఫీ రాయండి."
)

actor_prompt = PromptTemplate.from_template(
    "తెలుగు సినిమా నటుడు {person_name} గురించి ఒక చిన్న బయోగ్రఫీ రాయండి."
)

politician_prompt = PromptTemplate.from_template(
    "తెలుగు రాజకీయ నాయకుడు {person_name} గురించి ఒక చిన్న బయోగ్రఫీ రాయండి."
)

unknown_prompt = PromptTemplate.from_template(
    "{person_name} గురించి ఒక చిన్న బయోగ్రఫీ రాయండి."
)

# LLM క్రియేట్ చేయడం
llm = ChatOpenAI(model="gpt-3.5-turbo")

# ఔట్‌పుట్ పార్సర్ క్రియేట్ చేయడం
output_parser = StrOutputParser()

# పర్సన్ టైప్ డిటెక్టర్ ప్రాంప్ట్ క్రియేట్ చేయడం
detector_prompt = PromptTemplate.from_template(
    """ఈ వ్యక్తి {person_name} ఏ కేటగిరీకి చెందినవారు? క్రింది ఆప్షన్లలో ఒకటి మాత్రమే ఎంచుకోండి:
    - క్రికెట్ ఆటగాడు
    - సినిమా నటుడు
    - రాజకీయ నాయకుడు
    - తెలియదు
    
    జవాబు:
    """
)

# డిటెక్టర్ చెయిన్ క్రియేట్ చేయడం
detector_chain = detector_prompt | llm | output_parser

# కండిషనల్ బ్రాంచ్ క్రియేట్ చేయడం
branch = RunnableBranch(
    (lambda x: "క్రికెట్ ఆటగాడు" in x["category"], cricket_prompt | llm | output_parser),
    (lambda x: "సినిమా నటుడు" in x["category"], actor_prompt | llm | output_parser),
    (lambda x: "రాజకీయ నాయకుడు" in x["category"], politician_prompt | llm | output_parser),
    unknown_prompt | llm | output_parser
)

# ఫైనల్ చెయిన్ క్రియేట్ చేయడం
chain = {
    "person_name": lambda x: x["person_name"],
    "category": lambda x: detector_chain.invoke({"person_name": x["person_name"]})
} | branch

# వివిధ వ్యక్తుల కోసం చెయిన్‌ని రన్ చేయడం
print("క్రికెట్ ఆటగాడు:")
print(chain.invoke({"person_name": "VVS లక్ష్మణ్"}))

print("\nసినిమా నటుడు:")
print(chain.invoke({"person_name": "చిరంజీవి"}))

print("\nరాజకీయ నాయకుడు:")
print(chain.invoke({"person_name": "చంద్రబాబు నాయుడు"}))

## LCEL తో రీట్రైస్

LCEL మనకు ఫెయిల్యూర్ల నుండి రికవర్ చేయడానికి రీట్రై లాజిక్‌ని ఇంప్లిమెంట్ చేయడానికి అనుమతిస్తుంది:

In [ ]:
from langchain_core.runnables import RunnableRetry
import time

# ఫెయిల్ అయ్యే కంపోనెంట్ క్రియేట్ చేయడం
class UnreliableComponent(Runnable):
    """కొన్నిసార్లు ఫెయిల్ అయ్యే కంపోనెంట్."""
    
    def __init__(self, failure_rate=0.7):
        self.failure_rate = failure_rate
        self.attempts = 0
    
    def invoke(self, input: Dict[str, Any]) -> Dict[str, Any]:
        """కొన్నిసార్లు ఫెయిల్ అవుతుంది."""
        self.attempts += 1
        print(f"ప్రయత్నం {self.attempts}...")
        
        # సిమ్యులేటెడ్ ఫెయిల్యూర్
        if self.attempts < 3:  # మొదటి 2 ప్రయత్నాలు ఫెయిల్ అవుతాయి
            raise ValueError("కంపోనెంట్ ఫెయిల్ అయింది!")
        
        return {"result": f"విజయవంతంగా పూర్తయింది! ప్రయత్నం: {self.attempts}"}

# అన్‌రిలయబుల్ కంపోనెంట్‌ని క్రియేట్ చేయడం
unreliable = UnreliableComponent()

# రీట్రై లాజిక్‌తో కంపోనెంట్‌ని రాప్ చేయడం
retry_component = RunnableRetry(
    unreliable,
    retry_sleep=1,  # రీట్రైల మధ్య 1 సెకను వేచి ఉంటుంది
    max_retries=5   # గరిష్టంగా 5 రీట్రైలు
)

# రీట్రై లాజిక్‌తో కంపోనెంట్‌ని రన్ చేయడం
try:
    result = retry_component.invoke({"input": "టెస్ట్"})
    print(result)
except Exception as e:
    print(f"అన్ని రీట్రైలు ఫెయిల్ అయ్యాయి: {e}")

## LCEL తో రియల్-వరల్డ్ ఉదాహరణ: తెలుగు న్యూస్ క్లాసిఫికేషన్

ఇప్పుడు, మనం LCEL ని ఉపయోగించి ఒక రియల్-వరల్డ్ ఉదాహరణను చూద్దాం: తెలుగు న్యూస్ ఆర్టికల్స్‌ని క్లాసిఫై చేయడం.

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

# తెలుగు న్యూస్ ఆర్టికల్స్
telugu_news_articles = [
    """
    హైదరాబాద్, మే 15: ఐపీఎల్ 2024లో సన్‌రైజర్స్ హైదరాబాద్ జట్టు అద్భుతమైన ప్రదర్శనతో అభిమానులను ఆకట్టుకుంటోంది. గత మ్యాచ్‌లో లక్నో సూపర్ జెయింట్స్‌పై 10 వికెట్ల తేడాతో ఘన విజయం సాధించిన సన్‌రైజర్స్, పాయింట్ల పట్టికలో రెండో స్థానానికి చేరుకుంది.
    """,
    """
    హైదరాబాద్, మే 16: తెలంగాణ ప్రభుత్వం రాష్ట్రంలో కొత్త విద్యా విధానాన్ని ప్రవేశపెట్టనుంది. ఈ విధానంలో డిజిటల్ లెర్నింగ్‌కు ప్రాధాన్యత ఇవ్వనున్నారు. రాష్ట్ర విద్యా శాఖ మంత్రి మాట్లాడుతూ, కొత్త విద్యా విధానం ద్వారా విద్యార్థుల నైపుణ్యాలను మెరుగుపరచడమే లక్ష్యమని తెలిపారు.
    """,
    """
    హైదరాబాద్, మే 17: హైదరాబాద్‌లో జరిగిన తెలుగు సినిమా అవార్డుల వేడుకలో 'తిలక్' చిత్రం అత్యధిక అవార్డులను కైవసం చేసుకుంది. ఉత్తమ చిత్రం, ఉత్తమ దర్శకుడు, ఉత్తమ నటుడు సహా ఏడు కేటగిరీలలో అవార్డులు గెలుచుకుంది. ఈ వేడుకకు పలువురు ప్రముఖ సినీ ప్రముఖులు హాజరయ్యారు.
    """
]

# ఔట్‌పుట్ స్కీమా డిఫైన్ చేయడం
class NewsClassification(BaseModel):
    category: str = Field(description="న్యూస్ ఆర్టికల్ కేటగిరీ (క్రీడలు, రాజకీయాలు, విద్య, సినిమా, వ్యాపారం, ఆరోగ్యం, మరేదైనా)")
    summary: str = Field(description="న్యూస్ ఆర్టికల్ సారాంశం (1-2 వాక్యాలలో)")
    entities: List[str] = Field(description="న్యూస్ ఆర్టికల్‌లో ప్రస్తావించబడిన ముఖ్యమైన వ్యక్తులు, సంస్థలు లేదా స్థలాలు")
    sentiment: str = Field(description="న్యూస్ ఆర్టికల్ సెంటిమెంట్ (పాజిటివ్, నెగటివ్, న్యూట్రల్)")

# ఔట్‌పుట్ పార్సర్ క్రియేట్ చేయడం
parser = JsonOutputParser(pydantic_object=NewsClassification)

# ప్రాంప్ట్ టెంప్లేట్ క్రియేట్ చేయడం
prompt = PromptTemplate(
    template="""కింది తెలుగు న్యూస్ ఆర్టికల్‌ని విశ్లేషించి, దాని కేటగిరీ, సారాంశం, ముఖ్యమైన వ్యక్తులు/సంస్థలు మరియు సెంటిమెంట్‌ని గుర్తించండి.

న్యూస్ ఆర్టికల్: {article}

{format_instructions}
""",
    input_variables=["article"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# LLM క్రియేట్ చేయడం
llm = ChatOpenAI(model="gpt-3.5-turbo")

# LCEL ఉపయోగించి పైప్‌లైన్ క్రియేట్ చేయడం
chain = prompt | llm | parser

# ప్రతి న్యూస్ ఆర్టికల్‌ని క్లాసిఫై చేయడం
for i, article in enumerate(telugu_news_articles):
    print(f"న్యూస్ ఆర్టికల్ {i+1} క్లాసిఫికేషన్:")
    result = chain.invoke({"article": article})
    print(f"కేటగిరీ: {result['category']}")
    print(f"సారాంశం: {result['summary']}")
    print(f"ముఖ్యమైన వ్యక్తులు/సంస్థలు: {', '.join(result['entities'])}")
    print(f"సెంటిమెంట్: {result['sentiment']}")
    print("\n" + "-"*50 + "\n")

## ముగింపు

ఈ ట్యుటోరియల్‌లో, మనం LangChain Expression Language (LCEL) యొక్క అడ్వాన్స్డ్ ఫీచర్స్ గురించి నేర్చుకున్నాము:

1. LCEL తో మెమరీ ఇంటిగ్రేషన్ ఎలా చేయాలో చూశాము
2. LCEL తో కస్టమ్ కంపోనెంట్స్‌ని ఎలా క్రియేట్ చేయాలో నేర్చుకున్నాము
3. LCEL తో పారాలలైజేషన్ ఎలా ఇంప్లిమెంట్ చేయాలో చూశాము
4. LCEL తో కండిషనల్ లాజిక్ ఎలా ఇంప్లిమెంట్ చేయాలో నేర్చుకున్నాము
5. LCEL తో రీట్రై లాజిక్ ఎలా ఇంప్లిమెంట్ చేయాలో చూశాము
6. LCEL తో రియల్-వరల్డ్ ఉదాహరణను ఇంప్లిమెంట్ చేశాము

LCEL అనేది LangChain పైప్‌లైన్‌లను క్రియేట్ చేయడానికి శక్తివంతమైన మరియు ఎక్స్‌ప్రెసివ్ వే. ఇది కాంప్లెక్స్ LLM అప్లికేషన్‌లను సింపుల్ మరియు మెయింటెయిన్ చేయడానికి సులభమైన కోడ్‌తో బిల్డ్ చేయడానికి సహాయపడుతుంది.

తదుపరి ట్యుటోరియల్‌లో, మనం LangChain లో Runnable కాన్సెప్ట్ గురించి నేర్చుకుంటాము.